In [ ]:
# %pip install -r requirements.txt
# %pip install wandb
%pip install toml

In [ ]:
%pip install "unsloth[cu121-ampere] @ git+https://github.com/unslothai/unsloth.git"
# %pip install "unsloth[cu121_torch220] @ git+https://github.com/unslothai/unsloth.git"

In [1]:
import toml

# Reading from a TOML file
with open("config.toml", "r") as toml_file:
    training_config = toml.load(toml_file)

training_config['ModelArgs']['loftq_config'] = None

In [2]:
from lora import Lora_FineTuner


In [3]:
llm_trainer = Lora_FineTuner(dataset=training_config['dataset_path'],
                             model_name="unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
                            #  model_name = "unsloth/mistral-7b-v0.2-bnb-4bit",
                            max_seq_length=training_config['max_seq_length'],
                            load_in_4bit=training_config['load_in_4bit'],
                            model_args = training_config['ModelArgs'],
                            training_args = training_config['TrainingArguments']
                             )

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.24. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Unsloth 2024.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
llm_trainer.train(
                  resume = False,
                  # n_rows=250,
                 )

KeyError: "Column train not in the dataset. Current columns in the dataset: ['statement', 'relation', 'text']"

### Saving for VLLM

In [5]:
import datetime
now = datetime.datetime.now()

In [6]:
model_save_path_16bit = f"Cosmetics_Expert_{now.strftime('%Y-%m-%d_%H%M%S')}"

In [7]:
llm_trainer.model.save_pretrained_merged(model_save_path_16bit, llm_trainer.tokenizer, save_method = "merged_16bit",)

Unsloth: You have 2 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 8.69 out of 15.32 RAM for saving.


100%|██████████| 32/32 [00:49<00:00,  1.55s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving Cosmetics_Expert_2024-03-25_162452/pytorch_model-00001-of-00003.bin...
Unsloth: Saving Cosmetics_Expert_2024-03-25_162452/pytorch_model-00002-of-00003.bin...
Unsloth: Saving Cosmetics_Expert_2024-03-25_162452/pytorch_model-00003-of-00003.bin...
Done.


### Upload to S3

In [8]:
import boto3
import os

In [9]:
session = boto3.Session()

In [10]:
models_key = f"Models/{model_save_path_16bit}/"
model_to_upload = f"./{model_save_path_16bit}/"

In [11]:
for file in os.listdir(model_to_upload):
    file_path = os.path.join(model_to_upload, file)
    if os.path.isfile(file_path):
        # Upload the file to S3
        session.resource('s3').Bucket(bucket).Object(os.path.join(models_key, file)).upload_file(file_path)


In [ ]:
1+1